In [1]:
import os
import numpy as np
import cv2 as cv
from utils import Utils

In [2]:

menPath = "../Dataset_0-5/men/"
womenPath = "../Dataset_0-5/Women/"

In [3]:

def getThresholdedHand(frame):
    # Convert image to HSV
    hsvim = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    # Lower boundary of skin color in HSV
    lower = np.array([0, 48, 80], dtype="uint8")
    # Upper boundary of skin color in HSV
    upper = np.array([20, 255, 255], dtype="uint8")
    skinMask = cv.inRange(hsvim, lower, upper)

    # Gaussian filter (blur) to remove noise
    skinMask = cv.GaussianBlur(skinMask, (17, 17), 0)

    # get thresholded image
    # ret, thresh1 = cv.threshold(
    # skinMask, 100, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    thresh1 = cv.adaptiveThreshold(
        skinMask, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 355, 5)

    return thresh1

In [4]:

# # while True:
# class_dir = os.path.join(menPath, f"{0}")
# imgPath = os.path.join(class_dir, f'{0}_men ({1}).JPG')
# img = cv.imread(imgPath)
# img = getThresholdedHand(img)
# cv.imshow("img", img)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [166]:
def getMaskedHand2(frame):
        # Convert image to Lab
        rgpImg = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        labImg = cv.cvtColor(rgpImg, cv.COLOR_RGB2LAB)
        # The values of L,A and B channels are  mapped  in  the  range  from  0  up  to  255 
        meanL = np.mean(labImg[:, :, 0])
        meanA = np.mean(labImg[:, :, 1])
        meanB = np.mean(labImg[:, :, 2])
        stdL = np.std(labImg[:, :, 0])
        # stdB = np.std(labImg[:, :, 2])

        if meanA + meanB <=256:
                print("first")
                mask =labImg[:, :, 0] <= ( meanL - stdL/3)
                labImg[mask] = np.array([255, 128, 128])
                labImg[(~mask)] = np.array([0, 128, 128])
        else:
                print("second")
                mask1 = labImg[:, :, 0] <=  ( meanL )
                mask2 = labImg[:,:, 2] <=  (meanB)
                mask = mask1 & mask2
                labImg[mask] = np.array([255, 128, 128])#white
                labImg[(~mask)] = np.array([0, 128, 128])#black
        
        rgpImg2 = cv.cvtColor(labImg, cv.COLOR_LAB2RGB)
        NormalizedImg = cv.normalize(rgpImg2, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX)

        kernel = np.ones((13, 13), np.uint8)
        img_dilation = cv.dilate(NormalizedImg, kernel, iterations=1)
        img_erosion = cv.erode(img_dilation, kernel, iterations=1)
        # Gaussian filter (blur) to remove noise
        gaussImg = cv.GaussianBlur(img_erosion, (23, 23), 0)
        # get the white and black pixels 
        white_pixels = (gaussImg == np.array([1, 1, 1])).all(axis=2)
        black_pixels = (gaussImg == np.array([0, 0, 0])).all(axis=2)
        # get the average of the white and black pixels in the original img
        avgR_shadow = np.mean(rgpImg[:,:,0][white_pixels])#TODO: we should calculate the average of the shadow region
        avgG_shadow = np.mean(rgpImg[:,:,1][white_pixels])
        avgB_shadow = np.mean(rgpImg[:,:,2][white_pixels])
        avgR_without = np.mean(rgpImg[:,:,0][black_pixels])
        avgG_without = np.mean(rgpImg[:,:,1][black_pixels])
        avgB_without = np.mean(rgpImg[:,:,2][black_pixels])
        # replace the shadow pixels with the average of the shadow region
        # rgpImg[white_pixels] = np.array([avgR_shadow, avgG_shadow, avgB_shadow])
        # the ratio between the average of the shadow region and the average of the non-shadow region
        ratioR =avgR_shadow/avgR_without
        ratioG = avgG_shadow / avgG_without
        ratioB = avgB_shadow / avgB_without
        print(ratioR, ratioG, ratioB)

        rgpImg[white_pixels]= np.array([ratioR, ratioG, ratioB]) 
        # To eliminate the over-illuminated pixels
        rgpImg = cv.medianBlur(rgpImg, 17)
        return rgpImg

In [167]:
for i in range(1, 6):
    out = getMaskedHand2(cv.imread(f"D:/CMP/third_Year/second_semester/pattern Recognition/project/Dataset_0-5/men/2/2_men ({i}).JPG"))
    cv.imshow(f"shadow{i}",  cv.resize(out, (1024, 512)))
    # getMasked = Utils.getMaskedHand(out)
    # cv.imshow(f"maskedHand{i}", cv.resize(getMasked, (1024, 512)))
    cv.waitKey(0)
    cv.destroyAllWindows()

second
0.7596215385088455 0.7632772475252595 0.761414442934455
second
0.6404142345566991 0.6693096269486066 0.6840922610166067
second
0.658683990729366 0.682312850413912 0.6955177765119895
second
0.7148468251979377 0.7612649903291695 0.7841261485641784
second
0.5796254267089311 0.5671852519528883 0.5548968023699922
